In [7]:
# the library used
import numpy as np
import pandas as pd
import pysubgroup as ps
import re
from sklearn.feature_extraction import DictVectorizer


## Helper Function

In [2]:
def tags_preprocess(tags):
    """
    input: tag string
    output: list of individual tags in the given tag string
    function: preprocess a single tag string 
    """
    tags = tags.replace("'","")
    tags = tags.replace(" ","")
    tags = tags.replace("[","")
    tags = tags.replace("]","")
    tags = tags.split(",")
    tags = [x.lower() for x in tags]
    return tags

In [3]:
def ingredients_preprocess(dataframe):
    """
    input: dataframe
    output: list of distinct ingredient list
    function: preprocess the ingredient columns and return a list of distinct ingredients
    """
    distinct_ingredients = []
    
    for i in range(len(dataframe)):
        ingredients = dataframe.iloc[i]['ingredient']
        
        r = re.compile('[A-Z]{1}[a-zA-Z]+')
        ingredients = str(ingredients)
        #ingredients = ''.join(i for i in ingredients if not i.isdigit())
        ingredients = ingredients.replace("'","")
        ingredients = ingredients.replace(" ","")
        ingredients = ingredients.replace("[","")
        ingredients = ingredients.replace("]","")
        # remove text inside parentheses
        ingredients = re.sub(r'\([^())]*\)',"", ingredients)
        ingredients = ingredients.split(",")
        ingredients = list(filter(r.match, ingredients))
        ingredients = [x.lower() for x in ingredients]
        distinct_ingredients += ingredients
        dataframe.iloc[i]['ingredient'] = ingredients
        
    return list(set(distinct_ingredients))

In [4]:
def get_recipe_countries(countries, data):
    """
    input: list of countries, dataframe
    output: selected dataframe whose recipes is from these countries
    function: select the rows in dataframe whose "tag" value contain one country tag
    """
    # add a new column class 
    drop_index = []
    for i in range(len(data)):
        tags = data.loc[i]["tags"]
        tags = tags_preprocess(tags)
        
        country_same =[l for l in countries if l in tags]
            
        if len(country_same) == 1:
            data.at[i, 'label'] = country_same[0]
        if len(country_same) == 0:
            drop_index.append(i)
        if len(country_same) > 1:
            drop_index.append(i)
            #data.at[i, 'label'] = 'overlap'
            
    # drop the columns which has no season tags
    data = data.drop(data.index[drop_index])
    return data

In [5]:
def convert_to_dict(arr):
    """
    Helper function to convect an array of ingredients to a dictionary
    """
    d={}
    for a in arr:
        d[a]=1
    return d

In [8]:
# extract comment user dataset from original dataset
def extract_com_user(data):
    """
    input: dataframe
    output: a new dataframe with all the comment user information
    function: spilt the dictionary of the column 'comment user' in original dataset
    """
    df_com = pd.DataFrame()
    for index, item in data['comment_user'].iteritems():
        if (item != '[]'):
            if (item != 'no comment'):
                array = ast.literal_eval(item)
                df_array = pd.DataFrame(array)
                df_array['recipe_id'] = index
                df_com = pd.concat([df_com,df_array])
    return df_com

In [9]:
def sub_cat_in_com(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    list_sub_cat = []
    list_country = []
    df_com2 = pd.DataFrame()
    for index_1, item_1 in data['region'].iteritems(): 
        if (item_1 != None):
            list_sub = list(item_1)
            list_sub = ''.join(x for x in list_sub if x not in punct)
            list_sub_cat.append(list_sub)
    df_sub_cat = pd.DataFrame(list_sub_cat)
    df_sub_cat['region'] = df_sub_cat

    for index, item in data['country'].iteritems(): 
        if (item != None):
            list_cou = list(item)
            list_cou = ''.join(x for x in list_cou if x not in punct)

            list_country.append(list_cou)

    df_cou = pd.DataFrame(list_country)
    df_sub_cat['country'] = df_cou
    df_sub_cat['recipe_id'] = data['region'].index     
    df_com2 = pd.concat([df_com2,df_sub_cat])
    return df_com2

## Subgroup Discovery

In this section we will use subgroup discovery to explore the association rules between attributes

- why we choose subgroup discovery?

because we find out that subgroup discovery is quite powerful compared to other data mining techniques. As long as we set differnt target with different search space, we can use use subgroup discovery technique to dig almost all interesting pattern that we want to explore from the dataset. 


In [10]:
# read the data
data = pd.read_csv("all_data.csv")

In [11]:
data.head()

,categorize,recipe_name,tags,avg_score,difficulty,ingredient,rating_count,calorie,preparation_time,comment_user,recipe_url
0,Menüart,"""A bis Z""-Salat","['Salat', 'Gemüse', 'Sommer', 'Vegetarisch', '...",2.67,simpel,"['1m.-große', 'Zucchini', '1', 'Apfel,säuerlic...",1.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '24.09...",https://www.chefkoch.de/rezepte/23694213760282...
1,Menüart,"""Aufgehende Sonne""","['Asien', 'Hauptspeise', 'Nudeln', 'Beilage', ...",3.60,normal,"['500g', 'Nudeln(Mie-),ReisnudelnoderReis', '4...",3.0,500kcal,30 Min,"[{'rating': 'rating-5', 'comment_time': '10.08...",https://www.chefkoch.de/rezepte/18838313064020...
2,Regional,"""Black and White"" New York Cheesecake","['Backen', 'USAoderKanada', 'Kuchen']",0.00,normal,"['250g', 'Bitterschokolade,70%', '300g', 'Schl...",0.0,NaN,60 Min,NaN,https://www.chefkoch.de/rezepte/36158315436795...
3,Menüart,"""Bleib gesund""-Smoothie SuperNova","['Getränk', 'Vegetarisch', 'Frühstück', 'kalor...",4.03,simpel,"['1m.-große', 'Karotte(n)', '1', 'Apfel', '1St...",84.0,205kcal,5 Min,"[{'rating': 'rating-1', 'comment_time': '13.01...",https://www.chefkoch.de/rezepte/20464013311985...
4,Menüart,"""Bottermelk Anballerste""","['Hauptspeise', 'Europa', 'Suppe', 'gebunden',...",3.60,normal,"['500g', 'Kartoffel(n)', '1', 'Zwiebel(n)', '1...",3.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '10.11...",https://www.chefkoch.de/rezepte/33875015040182...


### Association rules between comment user information and recipe attributes

In [26]:
data.isnull()
# data_com = data['comment_user']
# df_com = extract_com_user(data_com)
# df_com.head()

,categorize,recipe_name,tags,avg_score,difficulty,ingredient,rating_count,calorie,preparation_time,comment_user,recipe_url,label
0,False,False,False,False,False,False,False,True,False,False,False,True
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
99307,False,False,False,False,False,False,False,True,False,False,False,True
99308,False,False,False,False,False,False,False,True,False,True,False,True
99309,False,False,False,False,False,False,False,True,False,True,False,True
99310,False,False,False,False,False,False,False,True,False,False,False,True


### Association rules among the recipe attributes

- the association with "ingredient" and "regions"

In [18]:
countries = ['china', 'indien', 'deutschland','frankreich','grossbritannien','österreich','usaoderkanada','italien','spanien',
             'portugal', 'japen','schweiz','türkei', 'thailand', 'russland', 'großbritannien & irland', 'vietnam', 'korea',
            'australien', 'ägypten', 'marokko', 'niederlande']

In [19]:
df_certain_countries = get_recipe_countries(countries, data)
df_certain_countries.reset_index(inplace = True)
df_certain_countries['label'].value_counts()

deutschland      4288
italien          3035
usaoderkanada    1410
österreich       1062
frankreich       1023
spanien           675
china             667
indien            628
thailand          559
türkei            501
portugal          357
schweiz           341
marokko           174
russland          162
niederlande       108
australien         93
korea              65
vietnam            56
ägypten            51
Name: label, dtype: int64

In [20]:
# drop all the columns other than "ingredient" and "label" column
columns_drop = ['index', 'categorize', 'recipe_name', 'tags', 'avg_score', 'difficulty','rating_count', 'calorie', 
                'preparation_time','comment_user', 'recipe_url']
df_certain_countries = df_certain_countries.drop(columns_drop, axis = 1)

# preprocess the ingredient column
distinct_ingredients = ingredients_preprocess(df_certain_countries)

# One hot encoding of the ingredients
df_certain_countries['ingredient'] = df_certain_countries.ingredient.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df_countries = pd.DataFrame(data = vectorizer.fit_transform(df_certain_countries.ingredient.tolist()), columns = distinct_ingredients)
new_df_countries['label'] = df_certain_countries.label

In [21]:
# apply 
new_df_countries

,ziegenweichkäse,balsamicodimodena,gurkensud,griebenschmalz,schokostreuseloderschokodekor,schokoladenchipszumtunken,penneoderanderenudeln,magerjoghurt,zuckerodereinfachzucker,hafersahne,...,presuntooderparmaschinkenindünnenscheiben,mandelinblättchen,koriandergrünoderthaibasilkumzumverzieren,rohrzucker/zucker,anchos,bergkäsezumbestreuen,traubensaftodersauser,pecorinoausamatrice,kaffir-limettenblatt-,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,thailand
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,usaoderkanada
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,deutschland
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,deutschland
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,usaoderkanada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,italien
15251,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,deutschland
15252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spanien
15253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,spanien
